In [1]:
'''Variables'''
RATIO = 0.5
SEGDIR = "/content/drive/MyDrive/110-1ComSem_final/tf_SegTexts"
# WORDLIST: 164個leveled word的詞表，type: list


In [2]:
!gdown --id 1sGu0ssLMvCNndS4G8yZtW8HXmBX7fMKS

Downloading...
From: https://drive.google.com/uc?id=1sGu0ssLMvCNndS4G8yZtW8HXmBX7fMKS
To: /content/MSW_leveled_list3.pkl
100% 444k/444k [00:00<00:00, 29.6MB/s]


In [3]:
'''Initializaing data structure MSW'''
!gdown --id 1_k1H_JyXwyRoRFcXa_YKGXgqxOg1Kx5a 

Downloading...
From: https://drive.google.com/uc?id=1_k1H_JyXwyRoRFcXa_YKGXgqxOg1Kx5a
To: /content/MSW.py
100% 642/642 [00:00<00:00, 949kB/s]


In [4]:
from MSW import MSW
import pickle

with open ('/content/MSW_leveled_list3.pkl', 'rb') as f:
   leveled_list = pickle.load(f)
# WORDLIST = []
# for msw in leveled_list:
# WORDLIST.append(msw.word)

In [5]:
import random 
import json 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def MakeDataset(target):
      '''
      input: target (string)
            source: SegTexts in tf_SegTexts (cleaned, stopword-removed)
      output: 
            train_dict: sense as keys, traintexts as values
            test_set: a list of tuple: (sent, sense) 
            category_dict: sense as keys, categories(hypernym) as values
      '''

      with open (f'{SEGDIR}/MSW_Segdict_{target}.json', 'r') as fp:
          target_dict = json.load(fp)
      category_dict = {}
      train_dict = {}
      test_set = []
      for sense, sense_text in target_dict.items():
            [coarse, fine] = sense.split('_')
            # 分類
            category_dict[sense] = coarse
            # randomize sense_text
            LEN = len(sense_text)
            # random.shuffle(sense_text) DISABLE random
            # 該dataset的性質是大概前面幾句會是summary
            sep = round(LEN*RATIO)
            if sep <= 0 and LEN >= 2: sep = 1
            # rand_ = random.sample(range(0, LEN), sep)
            X_train = [sense_text[i] for i in range(0, sep)]
            X_test = [sense_text[i] for i in range(sep, LEN)]
            
            for sent in X_test:
              '''只放有target word出現的'''
              if len(sent) > 0 and target in sent: 
                test_set.append((sent, sense))
            train_dict[sense] = X_train
      # print(test_set[0])
      return train_dict, test_set, category_dict 

### Utils


In [6]:
import numpy as np 
import pandas as pd 
def SavePredictions(idx: int, 
                    FILE: str,
                    testsent: list, 
                    y: str, 
                    yhat: str, 
                    df):
    ##### write new content using mode = 'a' ####
    with open (f'{FILE}', 'a') as f:
        
        f.write(f'\n* index: {idx}\n')
        f.write(f'\n* test sentence: \n')
        f.write(' '.join(testsent))
        f.write(f'\n* label: {y}\n')
        f.write(f'\n* prediction: {yhat}\n')
        f.write(f'\n* tfidf matrix: \n\n')
        # save matrix 
        # np.savetxt(f, df.values, fmt='%.5f')

In [8]:
'''
[USage]
for testsent in Xtest:
  CalcTFIDF()
'''
def CalcTFIDF(idx, 
              wsdword, 
              Xtraindict, 
              testcase, 
              CategoryDict):
      '''
      input:  idx: testcase編號
              wsdword: string (eg. '蘋果')
              Xtraindict: train_dict in MakeDataset()'s output
              testcase: a tuple, an instance of test_set in MakeDataset()'s output
              CategoryDict: sense as keys, categories(hypernym) as values, category_dict in MakeDataset()'s output
              FILE: 該wsd word之predictions寫入檔
      procedure: 
              calcs the tfidf dataframe 
              and the predicted yhat (string)
              and writes them into a file 
      return: 
            (y, yhat): tuple
      '''
      testsent = testcase[0]
      y = testcase[1]
      # print('calcTFIDF裡的testcase:', testcase)
      #############
      allsenses = Xtraindict.keys()
      docs = Xtraindict.values()
      yhat, tfidf_df = TFIDF(testsent, 
                             docs, 
                             allsenses, 
                             wsdword)
      return y, yhat, tfidf_df

In [9]:
'''
a_dataframe = pd.DataFrame([[1.1128, 2, 3],[4, 5/3, 10/3],[7, 8, 9]])
numpy_array = a_dataframe.to_numpy()
np.savetxt("test_file.txt", numpy_array, fmt = "%.4f")
'''

'\na_dataframe = pd.DataFrame([[1.1128, 2, 3],[4, 5/3, 10/3],[7, 8, 9]])\nnumpy_array = a_dataframe.to_numpy()\nnp.savetxt("test_file.txt", numpy_array, fmt = "%.4f")\n'

In [10]:
def RecordCoarse(y, yhat, WCoarseD):
    Cy_trues.append(WCoarseD[y])
    Cy_hats.append(WCoarseD[yhat])
    return WCoarseD[y] == (WCoarseD[yhat])
def RecordFineGrained(y, yhat):
    Fy_trues.append(y)
    Fy_hats.append(yhat)
    return y == yhat

### TESTING LOOP 

In [ ]:
def PRETTYFORMAT(df, FILE):
  with open (FILE, 'a') as FILEP:
      COLLEN = len(df.columns)
      COLWIDTH = 20 #chars
      
      print(' '*COLWIDTH, end = '\t', file = FILEP)
      for col in df.columns: 
            # print(col)
            print(f'{col}', end = '\t', file = FILEP)
      print('\n', file = FILEP)
      for rowidx, row in df.iterrows():
        spacelen = (COLWIDTH-len(rowidx))
        # print(spacelen) 幹中英文的width不等寬...
        spaces = ' '*spacelen
        print(rowidx, spaces, end = '\t', file = FILEP)
        for colname in df.columns:
            print(round(row[colname],4), end = '\t', file = FILEP)
        print('\n', file = FILEP)

In [11]:
'''Download tfidf utils'''
!gdown --id 1qFwOW8dOEMC2C5nXeOM2uttTMEtJQGAU

Downloading...
From: https://drive.google.com/uc?id=1qFwOW8dOEMC2C5nXeOM2uttTMEtJQGAU
To: /content/TFIDFutils.py
100% 1.80k/1.80k [00:00<00:00, 2.98MB/s]


In [12]:
%load_ext autoreload
%autoreload 2

In [13]:
from TFIDFutils import *

In [14]:
# /content/drive/MyDrive/110-1ComSem_final/tfidf_resources/Pred_ConMatrices
CONFDIR = '/content/drive/MyDrive/110-1ComSem_final/tfidf_resources/Pred_ConMatrices'
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
TOTAL_FAccCnt = 0
TOTAL_CAccCnt = 0
TOTAL_TSize = 0

In [ ]:
# dfstring = tfidfDF.to_string(max_colwidth = 40)
# np.savetxt('wow.txt', df.values, delimiter = '\t', fmt='%.5f')

'              Tfidf_sum   參見       眾議院        議員  總選舉   劇場   ==   參考   資料\nmain_刺客        0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0\n小說_刺客系列        0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0\n音樂_刺客樂團        0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0\n電影_刺客聯盟        0.000000  0.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0\n遊戲_刺客教條系列      0.000000  0.0  '

## 1/13 3:49 AM

In [20]:

import pandas as pd
W2VDIR = '/content/drive/MyDrive/110-1ComSem_final/w2v_resources'
DF = pd.read_csv(f'{W2VDIR}/msw_eda.csv')
Accurates = {'3 ~ 5':0, '< 3':0, '6 ~ 15':0}
Totals = {'3 ~ 5':0, '< 3':0, '6 ~ 15':0}
for i, W in enumerate(DF['target']):
    Cy_trues, Cy_hats = [], []
    WTrainD, WTestSet, WCategoryD  = MakeDataset(W)
    TESTSIZE = len(WTestSet)
    WCoarseL = list(set(WCategoryD.values()))
    WFineL = list(WCategoryD.keys())
    Type = DF.loc[i]['csense_cat']
    Totals[Type] += TESTSIZE
    for idx, testcase in enumerate(WTestSet):
        # Calc tfidf 
        y, yhat, tfidfDF= CalcTFIDF(idx = idx, 
              wsdword = W, 
              Xtraindict = WTrainD, 
              testcase = testcase, 
              CategoryDict = WCategoryD)
        CisCorrect = RecordCoarse(y, yhat, WCategoryD)
        if DF.loc[i]['csense#'] > 1:
            Accurates[Type] += CisCorrect 

--- Doc 29 of 斜拉 is missing.
--- Doc 29 of 跨徑 is missing.
--- Doc 29 of 懸索 is missing.
--- Doc 29 of 斜拉 is missing.
--- Doc 29 of 構造 is missing.
--- Doc 29 of 單塔 is missing.
--- Doc 29 of 雙塔 is missing.
--- Doc 29 of 單面 is missing.
--- Doc 29 of 布索 is missing.
--- Doc 29 of 布索 is missing.
--- Doc 29 of 密索 is missing.
--- Doc 29 of 少索 is missing.
--- Doc 29 of 形式 is missing.
--- Doc 29 of 佈置 is missing.
--- Doc 29 of 放射 is missing.
--- Doc 29 of 形式 is missing.
--- Doc 29 of 鉸接 is missing.
--- Doc 29 of 固接 is missing.
--- Doc 29 of 類型 is missing.
--- Doc 29 of 斜拉 is missing.
--- Doc 29 of 臺灣 is missing.
--- Doc 29 of 斜張 is missing.
--- Doc 29 of 德文 is missing.
--- Doc 29 of 斜索 is missing.
--- Doc 29 of 英文 is missing.
--- Doc 29 of "拉索 is missing.
--- Doc 29 of CableStayedBridge is missing.
--- Doc 29 of 根斜 is missing.
--- Doc 29 of 拉索 is missing.
--- Doc 29 of 形成 is missing.
--- Doc 29 of 斜拉 is missing.
--- Doc 29 of 孔樑 is missing.
--- Doc 29 of 斜拉索 is missing.
--- Doc 29 of 彈性 is missin

In [21]:
print(Accurates)
print(Totals)
keys = Totals.keys()
type_accuracy = {key:Accurates[key]/Totals[key] for key in keys}
type_accuracy
# TFiDF

{'3 ~ 5': 6308, '< 3': 777, '6 ~ 15': 2086}
{'3 ~ 5': 12410, '< 3': 1416, '6 ~ 15': 5127}


{'3 ~ 5': 0.5082997582594682,
 '6 ~ 15': 0.4068656134191535,
 '< 3': 0.548728813559322}

In [ ]:
PREDDIR = '/content/drive/MyDrive/110-1ComSem_final/tfidf_resources/predictions'
#### SummaryFile for all words #####
SUMMARY = f'{PREDDIR}/PredSummary.txt'
for W in WORDLIST: # WORDLIST 在tf_resources裡～ 
      # W as word of wsd target (string)
      Cy_trues, Cy_hats = [], []
      Fy_trues, Fy_hats = [], []
      WTrainD, WTestSet, WCategoryD  = MakeDataset(W)
      TESTSIZE = len(WTestSet)
      TOTAL_TSize += TESTSIZE
      
      print(f'[INFO] WSD_target: {W}')
      print(f'[INFO] TestSize: {TESTSIZE}')
      # CoarseSense的辭典
      WCoarseL = list(set(WCategoryD.values()))
      WFineL = list(WCategoryD.keys())
      
      PredFile = f'{PREDDIR}/{W}_pred.txt'
      
      # accuracy variables 
      CAccCount = 0
      FAccCount = 0
      for idx, testcase in enumerate(WTestSet):
            # Calc tfidf 
            y, yhat, tfidfDF= CalcTFIDF(idx = idx, 
                  wsdword = W, 
                  Xtraindict = WTrainD, 
                  testcase = testcase, 
                  CategoryDict = WCategoryD)
            SavePredictions(idx = idx, 
                      FILE = PredFile,
                      testsent = testcase[0],
                      y = y,
                      yhat = yhat, 
                      df = tfidfDF)
            # print('~~~~~pretty formatting')
            PRETTYFORMAT(tfidfDF, PredFile)

            # 這裡的y, yhat都是fine-grained sense 
            # Record accuracy 
            # print(f'[INFO]true, yhat:', y, yhat)
            CisCorrect = RecordCoarse(y, yhat, WCategoryD)
            FisCorrect = RecordFineGrained(y, yhat)
            CAccCount += CisCorrect
            FAccCount += FisCorrect 
      ##################
      ## write result ##
      with open(SUMMARY, 'w') as f:
        f.write(f'* TestSize:     {TESTSIZE}')
        if TESTSIZE <= 0:
           f.write(f'* No tests are carried out for {W}.')
           continue
        TOTAL_CAccCnt += CAccCount
        TOTAL_FAccCnt += FAccCount
        CAcc, FAcc = CAccCount/TESTSIZE, FAccCount/TESTSIZE
        f.write(f'* Coarse-grained Acc:   {CAcc:.4f}')
        f.write(f'* Fine-grained Acc:   {FAcc:.4f}')
        print(f'[INFO] CAcc: {CAcc:.4f}')
        print(f'[INFO] FAcc: {FAcc:.4f}')
        Cconf = confusion_matrix(Cy_trues, Cy_hats, labels = WCoarseL)
        Fconf = confusion_matrix(Fy_trues, Fy_hats, labels = WFineL)
      ### write confusion matrix 
        Cconf_string = np.array2string(Cconf)
        Fconf_string = np.array2string(Fconf)
        f.write(Cconf_string)
        f.write(Fconf_string)
      ## save confusion matrix visualization ###
        
      disp1 = ConfusionMatrixDisplay(Cconf, 
                                     display_labels = WCoarseL)
                          
      
      disp1.plot(xticks_rotation = 40)
      plt.gcf().subplots_adjust(bottom=0.5)
      plt.title(f'Coarse-grained CM: {W} | {CAcc:.4f}')
      plt.savefig(f'{CONFDIR}/{W}_CCM.png', dpi=150, bbox_inches='tight')
      disp2 = ConfusionMatrixDisplay(Fconf, display_labels = WFineL)
                                     
      disp2.plot(xticks_rotation =40)
      plt.title(f'Fine-grained CM: {W} | {FAcc:.4f}')
      plt.gcf().subplots_adjust(bottom=0.5)
      plt.savefig(f'{CONFDIR}/{W}_FCM.png', dpi=150, bbox_inches='tight')
      ### 分隔線 ###
      print(f'------------------')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(TOTAL_CAccCnt/TOTAL_TSize)
print(TOTAL_FAccCnt/TOTAL_TSize)

0.46649367044219237
0.2682961040782691


In [ ]:
TOTAL_TSize

28517

### Confusion Matrix Vis

In [ ]:
import matplotlib as mpl
import matplotlib.font_manager as fm
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download
fm.fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

--2021-12-25 04:20:08--  https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.google.com (drive.google.com)... 64.233.189.102, 64.233.189.113, 64.233.189.101, ...
Connecting to drive.google.com (drive.google.com)|64.233.189.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ohj2hmeq9l1qjuli14bjebkoc21gguu9/1640406000000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_ [following]
--2021-12-25 04:20:12--  https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ohj2hmeq9l1qjuli14bjebkoc21gguu9/1640406000000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving doc-0k-9o-docs.googleusercontent.com (doc-0k-9o-docs.googleusercontent.com)... 142.251.8.132, 2404:6800:4008:c15::84
Connecting to doc-0k-9o-docs.googleusercontent.com (doc-0k-9o-docs.googleusercontent.com)|142.251